# Terrain Visualization

In this activity, you will work with creating and manipulating 3D surface meshes using **PyVista**, a Python interface for the **Visualization Toolkit (VTK)**. VTK is a powerful open-source library for computer graphics, visualization, and image processing. You can learn more about both tools through these references:
- https://docs.pyvista.org/
- https://vtk.org/

We will also be using the **itkwidgets** library, which provides interactive Jupyter widgets for plotting, to visualize our meshes.

The outline of this activity will be:
1. Creating a 3D surface mesh
2. Writing code to coarsen the mesh
3. Writing code to visualize the error in elevation between the original mesh and the coarse mesh

# Part 1: Creating a 3D Surface Mesh
We will start by using a topographic surface to create a 3D terrain-following mesh.

Terrain following meshes are common in the environmental sciences, for instance
in hydrological modelling (see
[Maxwell 2013](https://www.sciencedirect.com/science/article/abs/pii/S0309170812002564)
and
[ParFlow](https://parflow.org)).

Below, we domonstrate a simple way to make a 3D grid/mesh that
follows a given topographic surface. In this example, it is important to note
that the given digital elevation model (DEM) is structured (gridded and not
triangulated): this is common for DEMs.


In [30]:
#grade (DO NOT DELETE THIS LINE)
import math
import numpy as np
import pylab as plt
import pyvista as pv
from pyvista import examples

In [2]:
%matplotlib widget
from pyvista import set_plot_theme
set_plot_theme('document')
pv.set_jupyter_backend('pythreejs')

Download a gridded topography surface (DEM) using one of the examples provided by PyVista.



In [3]:
dem = examples.download_crater_topo()
dem

UniformGrid (0x7f30b79d2ca0)
  N Cells:	1677401
  N Points:	1680000
  X Bounds:	1.810e+06, 1.831e+06
  Y Bounds:	5.640e+06, 5.658e+06
  Z Bounds:	0.000e+00, 0.000e+00
  Dimensions:	1400, 1200, 1
  Spacing:	1.500e+01, 1.500e+01, 0.000e+00
  N Arrays:	1

Now let's subsample and extract an area of interest to make this example
simple (also the DEM we just loaded is pretty big).
Since the DEM we loaded is a `pyvista.UniformGrid` mesh, we can use
the `pyvista.UniformGridFilters.extract_subset` filter to extract a 257x257-point (256x256-cell) area from the DEM:



In [4]:
subset = dem.extract_subset((572, 828, 472, 728, 0, 0), (1,1,1))
subset

UniformGrid (0x7f30b79d2d60)
  N Cells:	65536
  N Points:	66049
  X Bounds:	1.819e+06, 1.822e+06
  Y Bounds:	5.647e+06, 5.651e+06
  Z Bounds:	0.000e+00, 0.000e+00
  Dimensions:	257, 257, 1
  Spacing:	1.500e+01, 1.500e+01, 0.000e+00
  N Arrays:	1

Let's plot the area we just extracted to see what it looks like.

In [5]:
pv.plot(subset)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

Now that we have a region of interest for our terrain following mesh, lets
make a 3D surface of that DEM.

In [6]:
terrain = subset.warp_by_scalar() #Warp into a 3D surface mesh (without volume)
terrain

StructuredGrid (0x7f30b6e8f1c0)
  N Cells:	65536
  N Points:	66049
  X Bounds:	1.819e+06, 1.822e+06
  Y Bounds:	5.647e+06, 5.651e+06
  Z Bounds:	1.777e+03, 2.787e+03
  Dimensions:	257, 257, 1
  N Arrays:	1

We can see that our terrain is now a `pyvista.StructuredGrid` mesh. Now let's plot our terrain.

In [7]:
pv.plot(terrain)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

## Part 2: Coarsening the Mesh (and Writing Code)
In this section, you will write code to generate a new coarse mesh from our `terrain` mesh. Coarse meshes generally provide less accurate solutions, but are computationally faster. 

Your new mesh should be a `StructuredGrid`, just like the original mesh, but with a lower resolution. This means you will need to redefine the (x, y, z) coordinate points of your mesh. We will explain how to redefine your coordinates a little later on.

First, let's start with understanding how to generate a new mesh. You can initialize a new `StructuredGrid` object directly from the three point arrays that each contain the x, y, and z coordinates of all points in the mesh, respectively. Note: Each array is a 3D array with dimensions M x N x 1 (with the z-axis always being of length 1).

You will find the following reference helpful: https://docs.pyvista.org/version/stable/examples/00-load/create-explicit-structured-grid.html#sphx-glr-examples-00-load-create-explicit-structured-grid-py.

Let's look at the example below for initializing a new `StructuredGrid`.

In [8]:
xrng = np.arange(-10, 10, 2)                # [-10,  -8,  -6,  -4,  -2,   0,   2,   4,   6,   8]
yrng = np.arange(-10, 10, 2)
zrng = np.arange(-10, 10, 2)
x_example, y_example, z_example = np.meshgrid(xrng, yrng, zrng)
grid_example = pv.StructuredGrid(x_example, y_example, z_example)
grid_example

/opt/conda/lib/python3.8/site-packages/pyvista/utilities/helpers.py:475: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


StructuredGrid,Information
N Cells,729
N Points,1000
X Bounds,"-1.000e+01, 8.000e+00"
Y Bounds,"-1.000e+01, 8.000e+00"
Z Bounds,"-1.000e+01, 8.000e+00"
Dimensions,"10, 10, 10"
N Arrays,0


Now, let's follow the same general steps as in the above example to generate our new coarse mesh from our previously created `terrain` mesh.

We can coarsen the mesh by merging every `f` quads/cells into one and dropping the center point, where `f` is your sampling factor aka the factor by which you want to reduce the resolution. In other words, we can produce a reduced version of the mesh by sampling one out of every `f` points along each axis of the mesh.

Write code to coarsen `terrain` by a **factor of 2**. In other words, we will be converting the mesh from a 257x257-point mesh to a 129x129-point mesh (or equivalently, a 256x256-cell mesh to a 128x128-cell mesh). 

In the code block below, complete the implementation of `coarsen(terrain)` by defining three new point arrays, `xnew`, `ynew`, and `znew` and composing them into a new `StructuredGrid` object named `coarse`. Your function should return a tuple containing `(xnew, ynew, znew, coarse)`.
You will find the following reference helpful: https://docs.pyvista.org/version/stable/api/core/_autosummary/pyvista.StructuredGrid.html.

In [29]:
#grade (write your code in this cell and DO NOT DELETE THIS LINE)
#NOTE: You do not need to round any values within your results.
def coarsen(terrain):
    dims = terrain.dimensions
    x = terrain.x
    y = terrain.y
    z = terrain.z
    xnew = x[::2, ::2]
    ynew = y[::2, ::2]
    znew = z[::2, ::2]
    coarse = pv.StructuredGrid(xnew, ynew, znew)    
    return xnew, ynew, znew, coarse

    xnew, ynew, znew, coarse = coarsen(terrain)
    pv.plot(coarse)

# print(f"xnew shape: {xnew.shape}")
# print(f"ynew shape: {ynew.shape}")
# print(f"znew shape: {znew.shape}")

# # Print the values being checked in the assertions
# print(f"xnew[0][0][0]: {xnew[0][0][0]}")
# print(f"xnew[5][120][0]: {xnew[5][120][0]}")
# print(f"ynew[128][120][0]: {ynew[128][120][0]}")
# print(f"znew[12][12][0]: {znew[12][12][0]}")

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [25]:
### Tests for coarsenMesh. 
### Please DO NOT hard-code the answers as we will also be using hidden test cases when grading your submission.
    
print(f"Coarsening from {terrain.dimensions[0]} to {math.ceil(terrain.dimensions[0]/2)}...")
xnew, ynew, znew, coarse = coarsen(terrain)

assert xnew.shape == (129,129,1)
assert ynew.shape == (129,129,1)
np.testing.assert_allclose(xnew[0][0][0],1818580, rtol=1e-7)
np.testing.assert_allclose(xnew[5][120][0],1818730, rtol=1e-7)
np.testing.assert_allclose(ynew[128][120][0],5650680, rtol=1e-7)
np.testing.assert_allclose(znew[12][12][0],1880.53, rtol=1e-5)

Coarsening from 257 to 129...


We can plot the z-values of our new coarsened mesh by adding an additional attribute `values` to our mesh, which will contain a normalized, column-major flattened representation of the z-axis values of our grid.

See the following reference for more information on array flattening: https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flatten.html

In [11]:
#Plot the z-values using the viridis (default) color map
coarse['values'] = pv.plotting.normalize(coarse.z.flatten("F"))

pv.plot(coarse, scalars='values')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

## Part 3: Visualizing Error Values

Now that we have generated our coarse mesh, we can visualize the error in elevation between our coarse mesh and our original mesh. More specifically, we want to compute the error value for each point between the new (bilinearly interpolated) center point elevation and the original. We will then visualize the error as a scalar field on the original mesh.

Since we will need to bilinearly interpolate the center point elevation (i.e. the z-value) of each point in our coarse mesh in order to match the dimensions of our original mesh, let's define a function to do just that.

Define the function `bilin()` to bilinearly interpolate the value at coordinates `(x,y)` within a rectilinear grid of points.

**The parameters of your function are:**
- `x` = x-coordinate of point whose value we wish to interpolate
- `y` = y-coordinate of point whose value we wish to interpolate
- `points` = a list of four triplets of the form `(xc, yc, val)`, where `val` denotes the function value associated with coordinates `(xc, yc)`

This function should return a bilinearly interpolated value associated with coordinate `(x,y)` w.r.t the rectilinear grid formed by `points`.

**Hints:**
- You may assume the four triplets within `points` form a valid rectangle
- You may assume `x` and `y` fall within the rectangle formed by the `points` parameter
- You should NOT assume the four triplets within `points` are in any specific order

In [12]:
#grade (write your code in this cell and DO NOT DELETE THIS LINE)
#NOTE: You do not need to round any values within your results.
def bilin(x, y, points):
    points.sort(key=lambda element: (element[0], element[1]))
    x1, y1, q11 = points[0]
    x2, y2, q22 = points[3]
    q12 = points[1][2]
    q21 = points[2][2]

    if x1 == x2:
        return q11 * (y2 - y) / (y2 - y1) + q21 * (y - y1) / (y2 - y1)
    elif y1 == y2:
        return q11 * (x2 - x) / (x2 - x1) + q12 * (x - x1) / (x2 - x1)
    else:
        return (q11 * (x2 - x) * (y2 - y) +
                q21 * (x - x1) * (y2 - y) +
                q12 * (x2 - x) * (y - y1) +
                q22 * (x - x1) * (y - y1)) / ((x2 - x1) * (y2 - y1))
# testing_points = [(1, 1, 3), (3, 1, 6), (1, 3, 7), (3, 3, 9)]
# result = bilin(2, 2, testing_points)
# print(f"Result for (2, 2): {result}")
# result = bilin(2.5, 2.5, testing_points)
# print(f"Result for (2.5, 2.5): {result}")
# result = bilin(1.1, 1.1, testing_points)
# print(f"Result for (1.1, 1.1): {result}")

In [13]:
### Tests for bilin(x, y, points) function. 
### Please DO NOT hard-code the answers as we will also be using hidden test cases when grading your submission.
testing_points = [(1,1,3), (3,1,6), (1,3,7), (3,3,9)]
result = bilin(2,2,testing_points)
np.testing.assert_allclose(result,6.25, rtol=1e-2)
result = bilin(2.5,2.5,testing_points)
np.testing.assert_allclose(result,7.6875, rtol=1e-3)
result = bilin(1.1,1.1,testing_points)
np.testing.assert_allclose(result,3.3475, rtol=1e-3)

Now, using your `bilin()` function, create a new mesh or `StructuredGrid` object named `intmesh`, reconstructed from `coarse` using bilinear interpolation, with the same dimensions as our original mesh `terrain`. Your new mesh should contain the interpolated z-values for each point in `terrain`.

As a starting point, we have defined some of the variables that will be helpful to you for creating your new interpolated mesh. Specifically, we will be checking the values in `errz` and `intz`, as defined below:
- `intz`: a 3D array with the same shape as `terrain.z` that will contain the bilinearly interpolated z-values from the coarsened mesh.<br/>**Note:** `intz` is a 3D M x N x 1 array where the last dimension contains the z-values. You should note this when assigning elements to `intz`. The interpolated mesh is a 3D M by N by 1 array, so tests will fail if you assign values to it as if it were just a 2D M by N array, bypassing the z-axis entirely. So, if your code looks like `intz[x][y] = bilin(...)` the fix is to change it to something like `intz[x][y] = [ bilin(...) ]` or  `intz[x][y][0] = bilin(...)`.

- `errz`: a list of scalar values. This should contain the absolute error values between each z-value in the original mesh and each interpolated z-value in the new returned mesh

Just like how we added the attribute `values` to our coarse mesh in order to plot the z-values of the mesh, you should add an additional attribute `errors` to `intmesh` in order to plot the absolute error values between the z-values in the original mesh and the interpolated z-values in our new returned mesh.

In the code block below, complete the implementation of `reconstructMesh(terrain,coarse)`, which should return a tuple containing `(intz, errz, intmesh)`.

In [22]:
#grade (write your code in this cell and DO NOT DELETE THIS LINE)
#NOTE: You do not need to round any values within your results.
def reconstructMesh(terrain,coarse):
    errz   = [] #Create a new empty list for holding absolute error values
    intz   = np.zeros_like(terrain.z) #Create a new array for holding bilinearly interpolated values from coarse mesh
    scale  = (terrain.z.shape[0]-1)/(coarse.z.shape[0]-1) #Reduction factor between original and coarse; should equal 2
    for i in range(terrain.dimensions[0]):
        for j in range(terrain.dimensions[1]):
            x = i / scale
            y = j / scale

            x1 = int(np.floor(x))
            x2 = min(x1 + 1, coarse.dimensions[0] - 1)
            y1 = int(np.floor(y))
            y2 = min(y1 + 1, coarse.dimensions[1] - 1)

            points = [
                (x1 * scale, y1 * scale, coarse.z[x1, y1, 0]),
                (x2 * scale, y1 * scale, coarse.z[x2, y1, 0]),
                (x1 * scale, y2 * scale, coarse.z[x1, y2, 0]),
                (x2 * scale, y2 * scale, coarse.z[x2, y2, 0]),
            ]

            intz[i, j, 0] = bilin(i, j, points)
            errz.append(abs(intz[i, j, 0] - terrain.z[i, j, 0]))

    intmesh = pv.StructuredGrid(terrain.x, terrain.y, intz)
    intmesh['errors'] = errz
    return intz, errz, intmesh

    intz, errz, intmesh = reconstructMesh(terrain, coarse)
    intmesh.plot(scalars='errors')

# intz, errz, intmesh = reconstructMesh(terrain,coarse)
# print(intz[130][130][0])
# print(intz[247][13][0])
# print(errz[89])
# print(errz[30678])
# print(errz[-10])

In [23]:
### Tests for vizError. 
### Please DO NOT hard-code the answers as we will also be using hidden test cases when grading your submission.
intz, errz, intmesh = reconstructMesh(terrain,coarse)
np.testing.assert_allclose(intz[130][130][0],2547.8, rtol=1e-4)
np.testing.assert_allclose(intz[247][13][0],2142.71, rtol=1e-5)
np.testing.assert_allclose(errz[89],1.89996337890625, rtol=1e-2)
np.testing.assert_allclose(errz[30678],1.18499755859375, rtol=1e-2)
np.testing.assert_allclose(errz[-10],1.0299072265625, rtol=1e-2)

<ipython-input-12-c5a08a610bc7>:11: RuntimeWarning: invalid value encountered in double_scalars
  return q11 * (y2 - y) / (y2 - y1) + q21 * (y - y1) / (y2 - y1)


In [18]:
intmesh

StructuredGrid (0x7f30b4a9b7c0)
  N Cells:	65536
  N Points:	66049
  X Bounds:	1.819e+06, 1.822e+06
  Y Bounds:	5.647e+06, 5.651e+06
  Z Bounds:	1.778e+03, 2.780e+03
  Dimensions:	257, 257, 1
  N Arrays:	1

Now we can visualize the error values that we computed! We recommend adjusting the color map to better visualize the error values. You can change the color map by clicking the settings icon at the top left of the interface.

In [19]:
print(f"Visualizing error between resolutions {terrain.dimensions[0]} and {math.ceil(terrain.dimensions[0]/2)}...")

pv.plot(intmesh, scalars='errors')

Visualizing error between resolutions 257 and 129...


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

For reference, here is a sample of what your final visualization should look like (with the magma colormap applied):
<img src='error-visualization.png' width=600/>

As you can see the error values are represented in thin lines. One of the things that can be done to visualize the erroneous regions in a clearer way is to smooth the image. There are different methods for image smoothing, but here we use a simple one that updates the error value of each pixel as the average of the error of its surrounding pixels. For this purpose we want to use a $k \times k$ 2D convolution with a value of $\frac{1}{k^2}$ at each element. `scipy.signal` has an implementation of this function. The `errz` list that your `reconstructMesh` outputs is $1$-dimensional, so you would have to reshape it to the correct dimensions of your grid before giving that as input to the `convolve2d` function. You need to flatten this array again before returning that in the function below. For `convolve2d`, use the symmetrical boundary condition (see this for more info: 
[convolve2d](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html))

In [26]:
#grade (write your code in this cell and DO NOT DELETE THIS LINE)
#NOTE: You do not need to round any values within your results.
from scipy import signal

def smoothed_errors(errz, shape, kernel_size):
    errz_reshaped = np.reshape(errz, shape)
    kernel = np.ones((kernel_size, kernel_size)) / (kernel_size**2)
    smoothed = signal.convolve2d(errz_reshaped, kernel, boundary='symm', mode='same')
    return smoothed.flatten()

    shape = (terrain.z.shape[0], terrain.z.shape[1])
    np.random.seed(1)
    rand_errz = np.random.uniform(low=0.0, high=1.0, size=len(errz))
    smoothed_errz = smoothed_errors(rand_errz, shape, 4)

In [27]:
### Tests for smoothing the error. 
### Please DO NOT hard-code the answers as we will also be using hidden test cases when grading your submission.
shape = (terrain.z.shape[0], terrain.z.shape[1])
np.random.seed(1)
rand_errz = np.random.uniform(low=0.0, high=1.0, size=len(errz))
smoothed_errz = smoothed_errors(rand_errz, shape, 4)
np.testing.assert_allclose(smoothed_errz[95],0.5182084931710876, rtol=1e-2)
np.testing.assert_allclose(smoothed_errz[103],0.6119100598363368, rtol=1e-2)

Now we can use this function to smooth the error values before visualizing it.

In [ ]:
smoothed_errz = smoothed_errors(errz, shape, 2)
intmesh['errors'] = pv.plotting.normalize(smoothed_errz)
pl = pv.plot(intmesh, scalars='errors')

As we increase the kernel size, the error values become smoother and less informative about the specific regions with higher error values:

In [ ]:
smoothed_errz = smoothed_errors(errz, shape, 5)
intmesh['errors'] = pv.plotting.normalize(smoothed_errz)
pl = pv.plot(intmesh, scalars='errors')

One way to keep the regions with larger error values in such visualizations more noticeable is to apply a non-linear transformation to the error values. One such simple transformation is to raise all the values to the same power. If this value is larger than 1, the larger values increase more greatly compared to the smaller values. If the power is less than 1, the values become more similar to each other:

In [ ]:
smoothed_errz = smoothed_errors(errz, shape, 5)
transformed_errz = smoothed_errz**2
intmesh['errors'] = pv.plotting.normalize(transformed_errz)
pl = pv.plot(intmesh, scalars='errors')

In [ ]:
smoothed_errz = smoothed_errors(errz, shape, 5)
transformed_errz = smoothed_errz**0.5
intmesh['errors'] = pv.plotting.normalize(transformed_errz)
pl = pv.plot(intmesh, scalars='errors')